In [1]:
# Dependencies
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn'

import json
import numpy as np

import re

In [ ]:
from sqlalchemy import create_engine
import psycopg2

# pull in postgres password
import sys
sys.path.append("C:/Users/laura/bootcamp/")
from api_keys import db_password

# ZIP CODE DATA

In [2]:
# Store filepath in a variable
Zipcode = "Resources/Portland_Zip_and_Neighborhood.csv"

In [3]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zip_df = pd.read_csv(Zipcode, encoding="ISO-8859-1")
zip_df.head()

,Zip_Code,Neighborhood_Name
0,97212,Alameda
1,97217,Arbor Lodge
2,97222,Ardenwald
3,97230,Argay
4,97201,Arlington Heights


# AIRBNB DATA (no longer using but didn't want to delete code yet - Laura 4.7.22)

In [ ]:
# Store filepath in a variable
ARBNB_old = "Resources/AB_US_2020.csv"

In [ ]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df_old = pd.read_csv(ARBNB_old, encoding="ISO-8859-1")

In [ ]:
# Show just the header
#ARBNB_df_old.head()

In [ ]:
column_list = ARBNB_df_old.columns.values.tolist()
#column_list

In [ ]:
#ARBNB_df_old.describe()

In [ ]:
# Keep the rows that are Portland
ARBNB_Portland_filter_old=ARBNB_df_old.loc[(ARBNB_df_old["city"]=="Portland")]
#ARBNB_Portland_filter_old.head()

In [ ]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_filter_old.isna().sum()

In [ ]:
# drop columns that do not aid in analysis
ARBNB_Portland_old=ARBNB_Portland_filter_old.drop(columns=['ï»¿id','name','host_id','host_name',"neighbourhood_group","last_review"])
#ARBNB_Portland_old

In [ ]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_old.isna().sum()

In [ ]:
#decided to drop nan values from reviews_per_month, could also look at dropping column
#ARBNB_Portland_dropna_old=ARBNB_Portland_old.dropna()

In [ ]:
#ARBNB_Portland_dropna_old.isna().sum()

# AIRBNB DATA FROM AIRBNB SITE

In [ ]:
# Store filepath in a variable
ARBNB = "Resources/Portland_airbnb_InsideAirbnb.csv"

In [ ]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df = pd.read_csv(ARBNB, encoding="ISO-8859-1", 
                       #converters={"host_response_rate" : string_to_float} - error won't let do on N/A
                      )

In [ ]:
# Show just the header
ARBNB_df.head()

In [ ]:
ARBNB_df.dtypes

In [ ]:
column_list = ARBNB_df.columns.values.tolist()
column_list

In [ ]:
# keep columns to be used in analysis
ARBNB_df2=ARBNB_df[[
     #'id',
     'listing_url',
     #'scrape_id',
     #'last_scraped',
     #'name',
     #'description',
     #'neighborhood_overview',
     #'picture_url',
     #'host_id',
     #'host_url',
     #'host_name',
     #'host_since',
     #'host_location',
     #'host_about',
     'host_response_time',
     'host_response_rate',
     'host_acceptance_rate',
     'host_is_superhost',
     #'host_thumbnail_url',
     #'host_picture_url',
     #'host_neighbourhood',
     #'host_listings_count',
     #'host_total_listings_count',
     #'host_verifications',
     #'host_has_profile_pic',
     'host_identity_verified',
     #'neighbourhood',
     'neighbourhood_cleansed',
     #'neighbourhood_group_cleansed',
     'latitude',
     'longitude',
     #'property_type',
     'room_type',
     'accommodates',
     #'bathrooms' - unfortunately this is blank so bathrooms_text will need to be cleaned further,
     'bathrooms_text',
     'bedrooms',
     'beds',
     #'amenities',
     'price',
     'minimum_nights',
     'maximum_nights',
     #'minimum_minimum_nights',
     #'maximum_minimum_nights',
     #'minimum_maximum_nights',
     #'maximum_maximum_nights',
     #'minimum_nights_avg_ntm',
     #'maximum_nights_avg_ntm',
     #'calendar_updated',
     #'has_availability',  (could decide to keep availability in)
     #'availability_30',
     #'availability_60',
     #'availability_90',
     #'availability_365',
     #'calendar_last_scraped',
     #'number_of_reviews',
     #'number_of_reviews_ltm',
     'number_of_reviews_l30d',
     #'first_review',
     'last_review',
     'review_scores_rating',
     'review_scores_accuracy',
     'review_scores_cleanliness',
     'review_scores_checkin',
     'review_scores_communication',
     'review_scores_location',
     'review_scores_value',
     #'license',
     #'instant_bookable',
     #'calculated_host_listings_count',
     #'calculated_host_listings_count_entire_homes',
     #'calculated_host_listings_count_private_rooms',
     #'calculated_host_listings_count_shared_rooms',
     'reviews_per_month'
]]
pd.set_option('display.max_columns', None)
ARBNB_df2.head()

In [ ]:
#set new index - not needed with Katie's code on t and f
#ARBNB_df2_index=ARBNB_df2.set_index("id")

In [ ]:
#filter to only look at entire home/apt
ARBNB_Portland_filter=ARBNB_df2.loc[(ARBNB_df2["room_type"]=="Entire home/apt")]
ARBNB_Portland_filter

In [ ]:
# convert t/f and true/false to boolean int
def change_tf(value):
    if value == "t":
        return 1
    else:
        return 0
ARBNB_Portland_filter['host_is_superhost'] = ARBNB_Portland_filter['host_is_superhost'].apply(change_tf)
ARBNB_Portland_filter['host_identity_verified'] = ARBNB_Portland_filter['host_identity_verified'].apply(change_tf)
ARBNB_Portland_filter.head()

In [ ]:
#drop half-bath description since it is one data point and doesn't give a number
ARBNB_Portland_filter2=ARBNB_Portland_filter[ARBNB_Portland_filter.bathrooms_text != "Half-bath"]
ARBNB_Portland_filter2

In [ ]:
# Display an overview of the bathrooms_text column
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

In [ ]:
# Clean up bathrooms_text category. 
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

In [ ]:
# Clean up bathrooms_text category using iloc
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

In [ ]:
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

In [ ]:
# changes bathrooms_text to bathrooms
ARBNB_Portland_filter2.rename(columns = {'bathrooms_text':'bathrooms'}, inplace = True)
ARBNB_Portland_filter2

In [ ]:
ARBNB_Portland_filter2.isna().sum()

In [ ]:
ARBNB_Portland_dropna=ARBNB_Portland_filter2.dropna()
ARBNB_Portland_dropna.describe()

In [ ]:
# convert dates from object to date
ARBNB_Portland_dropna["last_review"]=ARBNB_Portland_dropna["last_review"].apply(pd.to_datetime)
ARBNB_Portland_dropna.head()

In [ ]:
ARBNB_Portland_dropna.isna().sum()

In [ ]:
ARBNB_Portland_dropna.dtypes

In [ ]:
## no longer needed used Katie's code
# #change host_is_superhost to boolean and host_identity_verified to boolean
# d={"t":True,"f":False}
# ARBNB_Portland_b1=ARBNB_Portland_dropna['host_is_superhost'].map(d)
# ARBNB_Portland_b1

In [ ]:
## no longer needed used Katie's code
# #change host_identity_verified to boolean
# ARBNB_Portland_b2=ARBNB_Portland_dropna['host_identity_verified'].map(d)

In [ ]:
## no longer needed used Katie's code
# join_df=pd.concat([ARBNB_Portland_dropna, ARBNB_Portland_b1],axis=1)
# join_df

In [ ]:
## no longer needed used Katie's code
# join_df.columns = [*join_df.columns[:-1], 'is_superhost']
# join_df

In [ ]:
## no longer needed used Katie's code
# join_df2=pd.concat([join_df, ARBNB_Portland_b2],axis=1)
# join_df2

In [ ]:
## no longer needed used Katie's code
# join_df2.columns = [*join_df2.columns[:-1], 'is_verified']
# join_df2

In [ ]:
## no longer needed used Katie's code
# join_df2.drop(columns=["host_is_superhost", "host_identity_verified"])

In [ ]:
#strip price of $ commma and .00 from price
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip("$")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.replace(",","")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip(".00")
ARBNB_Portland_dropna

In [ ]:
#strip host_response_rate of %
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].str.replace("%","")
ARBNB_Portland_dropna

In [ ]:
#strip host_acceptance_rate of %
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].str.replace("%","")
ARBNB_Portland_dropna

In [ ]:
#convert to float
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].astype(float)
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].astype(float)
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].astype(float)

In [ ]:
ARBNB_Portland_dropna.drop(columns=["room_type"])

In [ ]:
ARBNB_Portland_dropna.dtypes

In [ ]:
ARBNB_Portland_dropna.to_csv("Resources/cleaned_ARBNB.csv")

# ZILLOW DATA

In [4]:
# Store filepath in a variable
ZLLW = "Resources/portland_housing.csv"

In [5]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ZLLW_df = pd.read_csv(ZLLW, encoding="ISO-8859-1")

/Users/katiemtroll/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (20,50,161,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Show just the header
ZLLW_df.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,brokerageName,dateSold,daysOnZillow,description,favoriteCount,...,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,NaN,1.620000e+12,25.0,"2860 NE Blossom Hill Rd, Fairview, OR 97024 is...",NaN,...,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,Harcourts Real Estate Network Group,1.620000e+12,53.0,Cute as a button 3bed/2.5bath home with brand ...,NaN,...,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,ERA Freeman & Associates,1.620000e+12,11.0,What a wonderful large home at end of quiet cu...,NaN,...,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,"Premiere Property Group, LLC",1.620000e+12,11.0,NaN,1.0,...,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,NaN,1.620000e+12,14.0,"25-65 NW 20th St, Gresham, OR 97030 is a apart...",NaN,...,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568


In [7]:
column_list2 = ZLLW_df.columns.values.tolist()

column_list2

['ï»¿abbreviatedAddress',
 'address/city',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'brokerageName',
 'dateSold',
 'daysOnZillow',
 'description',
 'favoriteCount',
 'homeStatus',
 'homeType',
 'lastSoldPrice',
 'latitude',
 'livingArea',
 'longitude',
 'lotSize',
 'pageViewCount',
 'postingContact/name',
 'price',
 'priceHistory/0/attributeSource/infoString1',
 'priceHistory/0/attributeSource/infoString2',
 'priceHistory/0/attributeSource/infoString3',
 'priceHistory/0/buyerAgent/name',
 'priceHistory/0/buyerAgent/photo/url',
 'priceHistory/0/buyerAgent/profileUrl',
 'priceHistory/0/date',
 'priceHistory/0/event',
 'priceHistory/0/postingIsRental',
 'priceHistory/0/price',
 'priceHistory/0/priceChangeRate',
 'priceHistory/0/pricePerSquareFoot',
 'priceHistory/0/sellerAgent/name',
 'priceHistory/0/sellerAgent/photo/url',
 'priceHistory/0/sellerAgent/profileUrl',
 'priceHistory/0/showCountyLink',
 'priceHistory/0/source',
 'priceHistory/0/time',
 'priceHistory/1/attributeSource/i

In [8]:
ZLLW_df.dtypes

ï»¿abbreviatedAddress     object
address/city              object
address/zipcode            int64
bathrooms                float64
bedrooms                 float64
                          ...   
yearBuilt                float64
zestimate                float64
zestimateHighPercent     float64
zestimateLowPercent      float64
zpid                       int64
Length: 348, dtype: object

In [9]:
ZLLW_df2=ZLLW_df[[
    'ï»¿abbreviatedAddress',
     'address/city',
     'address/zipcode',
     'bathrooms',
     'bedrooms',
     #'brokerageName',
     #'dateSold', - not right format - will use priceHistory/0/date
     'daysOnZillow',
     #'description',
     #'favoriteCount',
     'homeStatus',
     'homeType',
     #'lastSoldPrice',  -   decided to just use priceHistory/0/price
     'latitude',
     'livingArea',
     'longitude',
     'lotSize',
     #'pageViewCount', - not enough data here
     #'postingContact/name',
     #'price',-   decided to just use priceHistory/0/price
     #'priceHistory/0/attributeSource/infoString1',
     #'priceHistory/0/attributeSource/infoString2',
     #'priceHistory/0/attributeSource/infoString3',
     #'priceHistory/0/buyerAgent/name',
     #'priceHistory/0/buyerAgent/photo/url',
     #'priceHistory/0/buyerAgent/profileUrl',
     'priceHistory/0/date',
     'priceHistory/0/event',
     #'priceHistory/0/postingIsRental',
     'priceHistory/0/price',
     #'priceHistory/0/priceChangeRate',
     'priceHistory/0/pricePerSquareFoot',
     #'priceHistory/0/sellerAgent/name',
     #'priceHistory/0/sellerAgent/photo/url',
     #'priceHistory/0/sellerAgent/profileUrl',
     #'priceHistory/0/showCountyLink',
     #'priceHistory/0/source',
     #'priceHistory/0/time',
     #'propertyTaxRate',
     'rentZestimate',
     #'resoFacts/atAGlanceFacts/0/factValue',
     #'resoFacts/basement', 
     #'resoFacts/heating/0',
     #'resoFacts/atAGlanceFacts/2/factLabel', just shows heating in every row - no value
     #'resoFacts/atAGlanceFacts/2/factValue',
     #'resoFacts/cooling/0',
     #'resoFacts/atAGlanceFacts/3/factLabel', just shows heating in every row - no value
     #'resoFacts/atAGlanceFacts/3/factValue',
     'taxAssessedValue',
     'taxAssessedYear',
     #'taxHistory/0/taxIncreaseRate',
     #'taxHistory/0/taxPaid',
     #'taxHistory/0/time',
     #'taxHistory/0/value',
     #'taxHistory/0/valueIncreaseRate',
     'url',
     'yearBuilt',
     #'zestimate',
     #'zestimateHighPercent',
     #'zestimateLowPercent',
     #'zpid'
]].copy()
pd.set_option('display.max_columns', None)
ZLLW_df2.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,daysOnZillow,homeStatus,homeType,latitude,livingArea,longitude,lotSize,priceHistory/0/date,priceHistory/0/event,priceHistory/0/price,priceHistory/0/pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,RECENTLY_SOLD,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,Sold,315400,175.0,1950.0,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,RECENTLY_SOLD,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,Sold,400000,264.0,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,Sold,512000,188.0,1995.0,397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,Sold,348000,286.0,1695.0,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,RECENTLY_SOLD,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,Sold,510000,213.0,1395.0,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [10]:
ZLLW_df3=ZLLW_df2.rename(columns={"ï»¿abbreviatedAddress": "Address", "address/city": "City","address/zipcode": "Zipcode",'priceHistory/0/price': "price",'priceHistory/0/date': "date_sold", 'priceHistory/0/pricePerSquareFoot': 'pricePerSquareFoot'})
ZLLW_df3.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeStatus,homeType,latitude,livingArea,longitude,lotSize,date_sold,priceHistory/0/event,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,RECENTLY_SOLD,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,Sold,315400,175.0,1950.0,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,RECENTLY_SOLD,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,Sold,400000,264.0,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,Sold,512000,188.0,1995.0,397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,RECENTLY_SOLD,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,Sold,348000,286.0,1695.0,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,RECENTLY_SOLD,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,Sold,510000,213.0,1395.0,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [11]:
#filter for only sold homes so we have accurate price
Zillow_Sold=ZLLW_df3.loc[(ZLLW_df3['priceHistory/0/event']=="Sold")]
Zillow_Sold['homeStatus'].value_counts()

RECENTLY_SOLD      25207
FOR_SALE              14
PRE_FORECLOSURE        2
FOR_RENT               2
FORECLOSED             1
Name: homeStatus, dtype: int64

In [12]:
Zillow_Sold2=Zillow_Sold.loc[(Zillow_Sold['homeStatus']=="RECENTLY_SOLD")]
Zillow_Sold2.head()
Zillow_Sold2['homeStatus'].value_counts()

RECENTLY_SOLD    25207
Name: homeStatus, dtype: int64

In [13]:
#drop homeStatus and priceHistory/0/event now that they both only show sold homes
zillow_df=Zillow_Sold2.drop(columns=['homeStatus', 'priceHistory/0/event'])
zillow_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,5/21/2021,315400,175.0,1950.0,273960.0,2020.0,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,53.0,SINGLE_FAMILY,45.547585,1518.0,-122.453224,3484.0,4/23/2021,400000,264.0,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,6/4/2021,512000,188.0,1995.0,397560.0,2020.0,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,6/4/2021,348000,286.0,1695.0,269520.0,2017.0,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,APARTMENT,45.511238,2400.0,-122.431534,8545.0,6/1/2021,510000,213.0,1395.0,421980.0,2020.0,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0


In [14]:
#determine how many home types there are to see if we need an other category - no other category needed less than 10 types
zillow_df['homeType'].value_counts()

SINGLE_FAMILY        21982
CONDO                 1523
TOWNHOUSE             1494
APARTMENT              193
HOME_TYPE_UNKNOWN       15
Name: homeType, dtype: int64

In [15]:
# convert dates from object to date
zillow_df["date_sold"]=zillow_df["date_sold"].apply(pd.to_datetime)
zillow_df.dtypes

Address                       object
City                          object
Zipcode                        int64
bathrooms                    float64
bedrooms                     float64
daysOnZillow                 float64
homeType                      object
latitude                     float64
livingArea                   float64
longitude                    float64
lotSize                      float64
date_sold             datetime64[ns]
price                          int64
pricePerSquareFoot           float64
rentZestimate                float64
taxAssessedValue             float64
taxAssessedYear              float64
url                           object
yearBuilt                    float64
dtype: object

In [16]:
#drop na and 0 for yearBuilt column
zillow2_df = zillow_df[zillow_df['yearBuilt'].notna()]
zillow3_df=zillow2_df.loc[(zillow2_df['yearBuilt']!=0)]

#change year built from float to datetime
zillow3_df.yearBuilt=pd.to_datetime(zillow3_df["yearBuilt"],format='%Y')

/Users/katiemtroll/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
#drop na and 0 for yearBuilt column
zillow4_df = zillow3_df[zillow3_df['taxAssessedYear'].notna()]
zillow5_df=zillow4_df.loc[(zillow4_df['taxAssessedYear']!=0)]

#change tax assessed year from float to datetime
zillow5_df.taxAssessedYear=pd.to_datetime(zillow5_df["taxAssessedYear"],format='%Y')
zillow5_df.dtypes
zillow5_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,2021-05-21,315400,175.0,1950.0,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,2021-06-04,512000,188.0,1995.0,397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,14.0,APARTMENT,45.511238,2400.0,-122.431534,8545.0,2021-06-01,510000,213.0,1395.0,421980.0,2020-01-01,https://www.zillow.com/homedetails/25-65-NW-20...,1967-01-01
5,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,7000.0,2021-06-01,404200,351.0,1800.0,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01


In [18]:
# check the values of all of the columns
col_cleaned=zillow5_df.columns.to_list()
print(col_cleaned)
print(zillow5_df.dtypes)

['Address', 'City', 'Zipcode', 'bathrooms', 'bedrooms', 'daysOnZillow', 'homeType', 'latitude', 'livingArea', 'longitude', 'lotSize', 'date_sold', 'price', 'pricePerSquareFoot', 'rentZestimate', 'taxAssessedValue', 'taxAssessedYear', 'url', 'yearBuilt']
Address                       object
City                          object
Zipcode                        int64
bathrooms                    float64
bedrooms                     float64
daysOnZillow                 float64
homeType                      object
latitude                     float64
livingArea                   float64
longitude                    float64
lotSize                      float64
date_sold             datetime64[ns]
price                          int64
pricePerSquareFoot           float64
rentZestimate                float64
taxAssessedValue             float64
taxAssessedYear       datetime64[ns]
url                           object
yearBuilt             datetime64[ns]
dtype: object


In [19]:
#check value counts of object columns that may be used in ML model
print(zillow5_df.City.value_counts())
print(zillow5_df.homeType.value_counts())

Portland        10964
Beaverton        2049
Hillsboro        1650
Gresham          1308
Lake Oswego      1113
Tigard           1104
Oregon City       995
Happy Valley      746
West Linn         679
Milwaukie         543
Sherwood          502
Tualatin          447
Aloha             365
Troutdale         289
Gladstone         223
Fairview          149
King City         110
Wood Village       42
Clackamas          24
Cornelius          12
Beavercreek         8
Damascus            6
Wilsonville         5
PORTLAND            5
Estacada            4
Newberg             2
Corbett             2
King city           2
Wood village        1
Boring              1
HAPPY VALLEY        1
Lake oswego         1
Molalla             1
Millwaukie          1
Name: City, dtype: int64
SINGLE_FAMILY        20704
CONDO                 1249
TOWNHOUSE             1205
APARTMENT              189
HOME_TYPE_UNKNOWN        7
Name: homeType, dtype: int64


The homeType values are good for ML. The cities need to be merged where like (ex. King City)

In [20]:
cities_zlw = zillow5_df.City.drop_duplicates().values.tolist()
cities_zlw_az = cities_zlw
cities_zlw_az.sort()
cities_zlw_az

['Aloha',
 'Beavercreek',
 'Beaverton',
 'Boring',
 'Clackamas',
 'Corbett',
 'Cornelius',
 'Damascus',
 'Estacada',
 'Fairview',
 'Gladstone',
 'Gresham',
 'HAPPY VALLEY',
 'Happy Valley',
 'Hillsboro',
 'King City',
 'King city',
 'Lake Oswego',
 'Lake oswego',
 'Millwaukie',
 'Milwaukie',
 'Molalla',
 'Newberg',
 'Oregon City',
 'PORTLAND',
 'Portland',
 'Sherwood',
 'Tigard',
 'Troutdale',
 'Tualatin',
 'West Linn',
 'Wilsonville',
 'Wood Village',
 'Wood village']

In [21]:
zillow5_df.City.replace(['HAPPY VALLEY'], 'Happy Valley', inplace=True)
zillow5_df.City.replace(['King city'], 'King City', inplace=True)
zillow5_df.City.replace(['Lake oswego'], 'Lake Oswego', inplace=True)
zillow5_df.City.replace(['Millwaukie'], 'Milwaukie', inplace=True)
zillow5_df.City.replace(['PORTLAND'], 'Portland', inplace=True)
zillow5_df.City.replace(['Wood village'], 'Wood Village', inplace=True)

In [22]:
cities_zlw_clean = zillow5_df.City.drop_duplicates().values.tolist()
cities_zlw_az_clean = cities_zlw_clean
cities_zlw_az_clean.sort()
cities_zlw_az_clean

['Aloha',
 'Beavercreek',
 'Beaverton',
 'Boring',
 'Clackamas',
 'Corbett',
 'Cornelius',
 'Damascus',
 'Estacada',
 'Fairview',
 'Gladstone',
 'Gresham',
 'Happy Valley',
 'Hillsboro',
 'King City',
 'Lake Oswego',
 'Milwaukie',
 'Molalla',
 'Newberg',
 'Oregon City',
 'Portland',
 'Sherwood',
 'Tigard',
 'Troutdale',
 'Tualatin',
 'West Linn',
 'Wilsonville',
 'Wood Village']

In [23]:
# zillow6_df=zillow5_df.rename(columns={'resoFacts/cooling/0': "Cooling", 'resoFacts/heating/0': "Heating","resoFacts/atAGlanceFacts/0/factValue": "Type",'resoFacts/basement': "Basement"})
# zillow6_df.head()

In [24]:
# #figure out how to clean column
# zillow6_df['Cooling'].value_counts()

In [25]:
zillow5_df.describe()

,Zipcode,bathrooms,bedrooms,daysOnZillow,latitude,livingArea,longitude,lotSize,price,pricePerSquareFoot,rentZestimate,taxAssessedValue
count,23354.000000,23313.000000,23037.000000,23354.000000,23354.000000,23328.000000,23354.000000,2.142700e+04,2.335400e+04,23327.000000,23161.000000,2.335200e+04
mean,97148.036182,2.549753,3.370838,186.173589,45.477603,2195.279321,-122.682745,2.656238e+04,5.892069e+05,281.021477,2853.358966,4.972209e+05
std,86.044957,1.009655,0.983783,113.273920,0.065504,1128.581859,0.143650,1.428475e+06,3.780916e+05,152.217663,1318.098700,3.267507e+05
min,97003.000000,0.000000,0.000000,1.000000,45.260349,0.000000,-123.074242,0.000000e+00,3.000000e+02,0.000000,995.000000,1.000000e+02
25%,97062.000000,2.000000,3.000000,80.000000,45.433059,1471.750000,-122.800117,4.791000e+03,4.000000e+05,224.000000,2181.000000,3.371750e+05
50%,97203.000000,3.000000,3.000000,188.000000,45.488197,2008.000000,-122.670502,6.969000e+03,5.250000e+05,264.000000,2555.000000,4.320110e+05
75%,97221.000000,3.000000,4.000000,293.000000,45.526180,2690.000000,-122.591402,1.001800e+04,6.650000e+05,318.000000,3170.000000,5.684700e+05
max,97267.000000,23.000000,43.000000,364.000000,45.646896,51290.000000,-122.280975,2.087221e+08,2.550000e+07,16558.000000,27029.000000,1.423671e+07


In [26]:
zillow5_df.isna().sum()

Address                  0
City                     0
Zipcode                  0
bathrooms               41
bedrooms               317
daysOnZillow             0
homeType                 0
latitude                 0
livingArea              26
longitude                0
lotSize               1927
date_sold                0
price                    0
pricePerSquareFoot      27
rentZestimate          193
taxAssessedValue         2
taxAssessedYear          0
url                      1
yearBuilt                0
dtype: int64

In [27]:
zillow5_df = zillow5_df.dropna()

In [28]:
zillow5_df.describe()

,Zipcode,bathrooms,bedrooms,daysOnZillow,latitude,livingArea,longitude,lotSize,price,pricePerSquareFoot,rentZestimate,taxAssessedValue
count,21088.000000,21088.000000,21088.000000,21088.000000,21088.000000,21088.000000,21088.000000,2.108800e+04,2.108800e+04,21088.000000,21088.000000,2.108800e+04
mean,97146.136571,2.572212,3.446225,187.772809,45.474906,2253.152219,-122.684485,2.670786e+04,6.028637e+05,279.238619,2911.396244,5.115755e+05
std,86.258279,0.988868,0.935209,112.938673,0.066103,1092.634008,0.144864,1.439889e+06,3.807551e+05,154.764770,1325.200837,3.191404e+05
min,97003.000000,0.000000,0.000000,1.000000,45.260349,320.000000,-123.074242,0.000000e+00,3.000000e+02,0.000000,1074.000000,9.500000e+02
25%,97062.000000,2.000000,3.000000,81.000000,45.429430,1529.000000,-122.803123,4.791000e+03,4.200000e+05,224.000000,2211.000000,3.519550e+05
50%,97202.000000,3.000000,3.000000,192.000000,45.484550,2064.000000,-122.669155,6.969000e+03,5.300000e+05,264.000000,2594.000000,4.431595e+05
75%,97221.000000,3.000000,4.000000,293.000000,45.524636,2734.000000,-122.591736,1.001800e+04,6.750000e+05,315.000000,3200.000000,5.775732e+05
max,97267.000000,22.500000,43.000000,364.000000,45.646896,51290.000000,-122.280975,2.087221e+08,2.550000e+07,16558.000000,27029.000000,1.423671e+07


#### Remove outliers for ML model
Primary outliers may exist in bed/bath, livingArea, lotSize.

In [29]:
#function to identify limits for outliers based on IQR
def outlier_treatment(datacolumn):
 sorted(datacolumn)
 Q1,Q3 = np.percentile(datacolumn , [25,75])
 IQR = Q3 - Q1
 lower_range = Q1 - (1.5 * IQR)
 upper_range = Q3 + (1.5 * IQR)
 return lower_range,upper_range

In [31]:
print(f'The outlier bounds for bathrooms are: {outlier_treatment(zillow5_df.bathrooms)}.')
print(f'The outlier bounds for bedrooms are: {outlier_treatment(zillow5_df.bedrooms)}.')
print(f'The outlier bounds for lotSize are: {outlier_treatment(zillow5_df.lotSize)}.')
print(f'The outlier bounds for livingArea are: {outlier_treatment(zillow5_df.livingArea)}.')

The outlier bounds for bathrooms are: (0.5, 4.5).
The outlier bounds for bedrooms are: (1.5, 5.5).
The outlier bounds for lotSize are: (-3049.5, 17858.5).
The outlier bounds for livingArea are: (-278.5, 4541.5).


In [32]:
# first, drop values < 0.
zillow5_df = zillow5_df.loc[(zillow5_df['lotSize']>0)]
zillow5_df['lotSize'].describe()

count    2.108300e+04
mean     2.671419e+04
std      1.440060e+06
min      2.000000e+00
25%      4.791000e+03
50%      6.969000e+03
75%      1.001800e+04
max      2.087221e+08
Name: lotSize, dtype: float64

In [33]:
zillow5_df = zillow5_df.loc[(zillow5_df['livingArea']>0)]
zillow5_df['livingArea'].describe()

count    21083.000000
mean      2253.182374
std       1092.620320
min        320.000000
25%       1529.000000
50%       2064.000000
75%       2734.000000
max      51290.000000
Name: livingArea, dtype: float64

In [34]:
# check the actual value counts for bedrooms and bathrooms
print(zillow5_df['bedrooms'].value_counts())
print(zillow5_df['bathrooms'].value_counts())

3.0     9698
4.0     6975
2.0     2073
5.0     1839
6.0      273
1.0      153
7.0       35
8.0       16
9.0        8
0.0        7
10.0       5
43.0       1
Name: bedrooms, dtype: int64
3.00     9014
2.00     6461
1.00     2759
4.00     1868
5.00      360
2.50      226
6.00      134
1.50      119
7.00       30
3.50       25
2.10       21
8.00       15
0.00       10
4.50        8
1.10        6
9.00        6
3.10        5
10.00       3
2.25        3
2.75        2
5.50        2
22.50       1
18.00       1
14.00       1
1.25        1
4.10        1
6.50        1
Name: bathrooms, dtype: int64


In [35]:
# drop 0 bed/bath
zillow5_df = zillow5_df.loc[(zillow5_df['bedrooms']>0)]
zillow5_df = zillow5_df.loc[(zillow5_df['bathrooms']>0)]
zillow5_df['bedrooms'].describe()

count    21070.000000
mean         3.447888
std          0.932897
min          1.000000
25%          3.000000
50%          3.000000
75%          4.000000
max         43.000000
Name: bedrooms, dtype: float64

In [36]:
# drop fraction baths not 1/2
zillow5_df = zillow5_df[zillow5_df['bathrooms'].mul(100).mod(100).astype(int).isin([0, 50])]
print(zillow5_df['bathrooms'].value_counts())

3.0     9014
2.0     6461
1.0     2756
4.0     1868
5.0      360
2.5      226
6.0      134
1.5      119
7.0       30
3.5       25
8.0       15
4.5        8
9.0        6
10.0       3
5.5        2
18.0       1
22.5       1
14.0       1
6.5        1
Name: bathrooms, dtype: int64


We are going to limit the data based on outlier bounds. Baths between 0.5 bath and 4.5 bath and bedrooms between 1.5 and 5.5. Living area and lot size will be between 0 for the lower bound and the calculated upper bound.

In [38]:
lb_bath,up_bath = outlier_treatment(zillow5_df.bathrooms)
z5_outliers=zillow5_df.loc[(zillow5_df["bathrooms"] <= up_bath) & (zillow5_df["bathrooms"] >= lb_bath)]

lb_bed,up_bed = outlier_treatment(zillow5_df.bedrooms)
z5_outliers=z5_outliers.loc[(z5_outliers["bedrooms"] <= up_bed) & (z5_outliers["bedrooms"] >= lb_bed)]

lb_ls,up_ls = outlier_treatment(zillow5_df.lotSize)
z5_outliers=z5_outliers.loc[(z5_outliers["lotSize"] <= up_ls) & (z5_outliers["lotSize"] > 0)]

lb_la,up_la = outlier_treatment(zillow5_df.livingArea)
z5_outliers=z5_outliers.loc[(z5_outliers["livingArea"] <= up_la) & (z5_outliers["livingArea"] > 0)]

z5_outliers.describe()

,Zipcode,bathrooms,bedrooms,daysOnZillow,latitude,livingArea,longitude,lotSize,price,pricePerSquareFoot,rentZestimate,taxAssessedValue
count,18265.000000,18265.000000,18265.000000,18265.000000,18265.000000,18265.000000,18265.000000,18265.000000,1.826500e+04,18265.000000,18265.000000,1.826500e+04
mean,97148.242431,2.460827,3.368191,187.579250,45.479446,2078.627922,-122.685430,6852.924884,5.496832e+05,275.870298,2708.264166,4.636757e+05
std,86.271481,0.814036,0.769367,112.751753,0.063112,765.683850,0.145195,3171.615991,2.242703e+05,85.691069,839.458385,1.899900e+05
min,97003.000000,1.000000,2.000000,1.000000,45.300133,528.000000,-123.031128,421.000000,3.000000e+02,0.000000,1160.000000,9.500000e+02
25%,97062.000000,2.000000,3.000000,82.000000,45.436565,1498.000000,-122.806870,4791.000000,4.100000e+05,225.000000,2195.000000,3.449300e+05
50%,97203.000000,3.000000,3.000000,192.000000,45.489208,1978.000000,-122.666969,6534.000000,5.170000e+05,264.000000,2527.000000,4.260800e+05
75%,97221.000000,3.000000,4.000000,293.000000,45.526108,2561.000000,-122.592979,8712.000000,6.335000e+05,313.000000,2995.000000,5.393100e+05
max,97267.000000,4.500000,5.000000,364.000000,45.646259,4540.000000,-122.291786,17800.000000,4.000000e+06,2019.000000,14203.000000,4.008660e+06


In [40]:
zillow5_df = z5_outliers

In [44]:
print(f'The max for bathrooms is {zillow5_df.bathrooms.max()} and the min for bathrooms is {zillow5_df.bathrooms.min()}.')
print(f'The max for bedrooms is {zillow5_df.bedrooms.max()} and the min for bedrooms is {zillow5_df.bedrooms.min()}.')
print(f'The max for lotSize is {zillow5_df.lotSize.max()} and the min for lotSize is {zillow5_df.lotSize.min()}.')
print(f'The max for livingArea is {zillow5_df.livingArea.max()} and the min for livingArea is {zillow5_df.livingArea.min()}.')

The max for bathrooms is 4.5 and the min for bathrooms is 1.0.
The max for bedrooms is 5.0 and the min for bedrooms is 2.0.
The max for lotSize is 17800.0 and the min for lotSize is 421.0.
The max for livingArea is 4540.0 and the min for livingArea is 528.0.


In [45]:
zillow5_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,2021-05-21,315400,175.0,1950.0,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,2021-06-04,512000,188.0,1995.0,397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01
3,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
5,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,7000.0,2021-06-01,404200,351.0,1800.0,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01
6,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,SINGLE_FAMILY,45.498184,2036.0,-122.451332,6969.0,2021-05-21,478200,235.0,2250.0,373030.0,2020-01-01,https://www.zillow.com/homedetails/110-NW-Will...,1978-01-01


In [46]:
zillow5_df.to_csv("Resources/cleaned_zillow.csv", index=False)